# Import von WIAG-Daten in FactGrid
Formatiere CSV-Daten aus WIAG als V1-Statements für QuickSteps  
siehe [Einführung in QuickStatements](https://www.wikidata.org/wiki/Help:QuickStatements)
und die [Properties des FactGrid](https://database.factgrid.de/wiki/FactGrid:Directory_of_Properties)

Änderungen

- *2023-12-11* Verwende die Gruppe des höchsten erreichten Amtes in der englischen Beschreibung und für P165

Inhalt:

- [neue Objekte in FactGrid anlegen](#neue-Objekte-in-FactGrid-anlegen)

In [52]:
using CSV, DataFrames, Dates

In [4]:
ENV["COLUMNS"] = 120

120

In [5]:
input_path = "C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

In [6]:
output_path = input_path

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

## neue Objekte in FactGrid anlegen
Es wird davon ausgegangen, dass die betreffenden Personen noch nicht in FactGrid vorhanden sind. Die Spalte "FactGrid_ID" sollte also für alle Einträge leer sein.

Daten aus WIAG einlesen: Quelle Export "CSV Personendaten"

In [15]:
input_file = "WIAG-can_2023-12-13_ii.csv"
input_path_file = joinpath(input_path, input_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid\\WIAG-can_2023-12-13_ii.csv"

In [16]:
df_person_all = CSV.read(input_path_file, DataFrame);
nrow(df_person_all)

15144

Betrachte nur Neu-Zugänge (keine FactGrid-ID)

In [17]:
df_person = subset(df_person_all, :FactGrid_ID => ByRow(ismissing));
nrow(df_person)

13878

Lies die Amtsdaten ein (zunächst für P1018)

In [32]:
input_file = "WIAG-can-roles_2023-12-13.csv"
input_path_file = joinpath(input_path, input_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid\\WIAG-can-roles_2023-12-13.csv"

In [33]:
df_role_all = CSV.read(input_path_file, DataFrame);
nrow(df_role_all)

35426

Benenne die (relevanten) Spalten um entsprechend den Konventionen des FactGrid

In [35]:
names(df_role_all)

13-element Vector{String}:
 "person_id"
 "id"
 "name"
 "role_group"
 "role_group_en"
 "role_group_fq_id"
 "institution"
 "diocese"
 "date begin"
 "date end"
 "GND"
 "GSN"
 "FactGrid"

In [36]:
columns = [
    :person_id => :person_id,
    :role_group_fq_id => :P1018
]

2-element Vector{Pair{Symbol, Symbol}}:
        :person_id => :person_id
 :role_group_fq_id => :P1018

In [37]:
df_role = select(df_role_all, columns);

Es werden die Ämter eingetragen, die zur Gruppe "Oberstes Leitungsamt Diözese" oder "Leitungsamt Domstift" gehören.

In [56]:
is_relevant(s) = !ismissing(s) && s in ["Q648326", "Q648232"]

is_relevant (generic function with 1 method)

In [58]:
df_role = subset(df_role, :P1018 => ByRow(is_relevant));
nrow(df_role)

4793

Das deutsche Beschreibungsfeld enthält Lebensdaten mit der Zusammenfassung der Amtsdaten

In [18]:
join_komma(a, b) = join((a, b), ", ")
transform!(df_person, [:biographical_dates, :summary_offices] => ByRow(join_komma) => :description_de);

Das FactGrid arbeitet für Wikipedia nicht mit URL-codierten Adressen.
*2023-12-13* WIAG gibt direkt unkodierte URLs aus, daher sind die folgenden Schritte auskommentiert

In [9]:
# unescape_not_mg(s) = ismissing(s) ? s : URIs.unescapeuri(s)

unescape_not_mg (generic function with 1 method)

In [41]:
# transform!(df_person, :Wikipedia => ByRow(unescape_not_mg) => :Wikipedia);

In [19]:
columns = [:givenname, :prefix, :familyname, :Wikipedia]
dropmissing(df_person, :Wikipedia)[5:9, columns]

Row,givenname,prefix,familyname,Wikipedia
,String,String31?,String?,String
1,Wolfgang,Pfalzgraf von,Bayern,Wolfgang_(Pfalz-Neumarkt)
2,Johann Jakob,von,"Königsegg zu Rothenfels, Aulendorf und Staufen",Johann_Jacob_von_Königsegg-Rothenfels
3,Ernst Friedrich,von,Ascheberg,Ernst_Friedrich_von_Ascheberg_zu_Venne
4,August Georg Simpert,Markgraf von,Baden-Baden und Hochberg,August_Georg_Simpert
5,Christian,von,Hessen-Rotenburg,Christian_von_Hessen-Rotenburg


In [20]:
names(df_person)

19-element Vector{String}:
 "id"
 "corpus"
 "givenname"
 "prefix"
 "familyname"
 "displayname"
 "date_of_birth"
 "date_of_death"
 "biographical_dates"
 "summary_offices"
 "career_factgrid_id"
 "career"
 "career_en"
 "GND_ID"
 "GSN"
 "FactGrid_ID"
 "Wikidata_ID"
 "Wikipedia"
 "description_de"

Benenne die Spalten um entsprechend den Konventionen des FactGrid  
*2023-12-14* `career` wird jetzt mehrfach anhang der Amtsdaten eingetragen.

In [21]:
columns = [
    :displayname => :Lde,
    :description_de => :Dde,
    :date_of_birth => :P77,
    :date_of_death => :P38,
    :GND_ID => :P76,
    :GSN => :P472,
    :id => :P601,
    :Wikidata_ID => :Swikidatawiki,
    :Wikipedia => :Sdewiki
];

In [22]:
rename!(df_person, columns);

Kopiere das Label in Deutsch für die anderen Sprachen

In [23]:
df_person.Len = df_person.Lde;
df_person.Lfr = df_person.Lde;
df_person.Les = df_person.Lde;

Erstelle eine Beschreibung für Englisch aus Lebensdaten und der Gruppe des höchsten Amtes (Spalte 'career_en')

In [24]:
join_komma(a, b) = join([a, b], ", ")

join_komma (generic function with 1 method)

In [26]:
transform!(df_person, [:biographical_dates, :career_en] => ByRow(join_komma) => :Den);

In [39]:
df_person[11:19, [:P601, :givenname, :prefix, :familyname, :Len, :Den]]

Row,P601,givenname,prefix,familyname,Len,Den
,String31,String,String31?,String?,String,String
1,WIAG-Pers-CANON-13447-001,Hertnid,vom,Stein zu Ostheim,Hertnid vom Stein zu Ostheim,"+ 1491, Dignitary of a cathedral chapter"
2,WIAG-Pers-CANON-26352-001,Melchior,missing,Truchseß von Pommersfelden,Melchior Truchseß von Pommersfelden,"+ 1493/1494, Dignitary of a cathedral chapter"
3,WIAG-Pers-CANON-88038-001,Heinrich,missing,Nithart,Heinrich Nithart,"+ 1500, Dignitary of a cathedral chapter"
4,WIAG-Pers-CANON-83557-001,Pankraz,missing,Meusl,Pankraz Meusl,"~ 1482, Cleric of a cathedral chapter"
5,WIAG-Pers-CANON-24824-001,Casimir,von,Brandenburg,Casimir von Brandenburg,"* 1481, + 1527, Cleric of a cathedral chapter"
6,WIAG-Pers-CANON-90006-001,Sigismund,von,Pflug,Sigismund von Pflug,"+ 1510, Dignitary of a cathedral chapter"
7,WIAG-Pers-CANON-24792-001,Georg,von,Bibra,Georg von Bibra,"+ 1536, Dignitary of a cathedral chapter"
8,WIAG-Pers-CANON-80952-001,Christoph,von,Knöringen,Christoph von Knöringen,"~ 1499, Dignitary of a cathedral chapter"
9,WIAG-Pers-CANON-84019-001,Friedrich,von,Leonrod,Friedrich von Leonrod,"* 1483, + 1539, Cleric of a cathedral chapter"


Füge Daten ein, die für alle Personen gleich sind:
Mensch, Teil der Germania Sacra Forschungsdaten, männlich

In [28]:
insertcols!(df_person, 
    :P2 => "Q7",
    :P131 => "Q153178",
    :P154 => "Q18"
);    

Definiere Umwandlungsfunktionen  
*offen:* Datumsangaben sind noch zu besprechen, wegen der Verarbeitung von unscharften Zeitangaben.

In [29]:
fqs_string(s) = "\"" * string(s) * "\""

fqs_string (generic function with 1 method)

In [30]:
function create(out, row, col_list) 
    println(out, "CREATE")    
    for col in col_list
        println(out, "LAST\t", string(col), "\t", fqs_string(row[col]))        
    end
end

create (generic function with 1 method)

In [31]:
"""
    set_property_list (out, row, object, col_list, fmt_list)

    

# Arguments
- `out::IOStream`: output stream
- `row::DataFrameRow`: input data
- `object`: FactGrid identifier or "LAST"
- `col_list`: list of colums where data should be read from
- `fmr_list`: list of formatting functions (one for each column)
"""
function set_property_list(out, row, object, col_list, fmt_list)
    for (col, fmt) in zip(col_list, fmt_list)
        if (!ismissing(row[col]))
            println(out, object, "\t", string(col), "\t", fmt(row[col]))
        end
    end
end

set_property_list

Lies die Amtsdaten für einen bestimmten Domherren

In [59]:
"""
    set_role(df_role, id)

extract roles for person with `id`
"""
function set_role(out, df_role, id, object, property)
    property = Symbol(property)
    df_pr = subset(df_role, :person_id => ByRow(isequal(id)));
    df_pr = subset(df_pr, )
    fq_id_list = unique(df_pr[!, property])
    for fq_id in fq_id_list
        println(out, object, "\t", property, "\t", fq_id)
    end
    return nothing
end


set_role

Gib ausgewählte Elemente aus `df_person` aus. Falls es schon eine Datei mit gleichem Namen im angegebenen Verzeichnis gibt, wird die Datei überschrieben.

In [53]:
date_key = Dates.format(now(), "yyyy-mm-dd")

"2023-12-14"

In [60]:
output_file = "Insert_WIAG-can_" * date_key * "_FQ.csv"
output_path_file = joinpath(output_path, output_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid\\Insert_WIAG-can_2023-12-14_FQ.csv"

In [61]:
open(output_path_file, "w") do out_stream
    for row in eachrow(df_person)
        create(out_stream, row, [:Lde, :Len, :Lfr, :Les, :Dde, :Den])
        set_property_list(out_stream, row, "LAST", 
            [:P2, :P131, :P154, :P601, :P76, :P472, :Swikidatawiki, :Sdewiki], 
            [identity, identity, identity, fqs_string, fqs_string, fqs_string, fqs_string, fqs_string])
        set_role(out_stream, df_role, row[:P601], "LAST", :P1018)
    end        
end